In [19]:
!pip list

Package                  Version
------------------------ -----------
aiohappyeyeballs         2.6.1
aiohttp                  3.11.16
aiosignal                1.3.2
annotated-types          0.7.0
anyio                    4.9.0
appnope                  0.1.4
asttokens                3.0.0
attrs                    25.3.0
beautifulsoup4           4.13.3
bs4                      0.0.2
certifi                  2025.1.31
charset-normalizer       3.4.1
click                    8.1.8
colorama                 0.4.6
comm                     0.2.2
dataclasses-json         0.6.7
debugpy                  1.8.14
decorator                5.2.1
distro                   1.9.0
duckduckgo_search        8.0.0
executing                2.2.0
frozenlist               1.5.0
h11                      0.14.0
httpcore                 1.0.8
httpx                    0.28.1
httpx-sse                0.4.0
icecream                 2.1.4
idna                     3.10
ipykernel                6.29.5
ipython             

# model

In [ ]:
# import environment variables
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

In [38]:
ZHIPU_API_KEY = os.getenv("ZHIPU_API_KEY")
# print(ZHIPU_API_KEY)

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="glm-4-flash",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/",
    openai_api_key=ZHIPU_API_KEY,
    temperature=0.1,
)

In [40]:
model.invoke("你好，世界！")

AIMessage(content='你好！很高兴见到你，有什么可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 9, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'id': '202505171145307d88fefed0b64537', 'finish_reason': 'stop', 'logprobs': None}, id='run-dbd518c9-0e3a-4129-bdbc-b543767ce5fb-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22, 'input_token_details': {}, 'output_token_details': {}})

# tools

## search


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
import pandas as pd
from pandasql import sqldf


def simulate_database_operation(sql):
    my_table = pd.DataFrame(
        {
            "name": [
                "Henry Myers",
                "Martha Hawkins",
                "Kelsey Lutz",
                "Jonathan Fowler",
                "Jonathan Young",
                "Autumn Johnson",
                "Kimberly Macias",
                "Jared Mccormick",
                "Casey Hoover",
                "Erica Morse",
            ],
            "age": [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
            "sex": ["F", "M", "M", "F", "F", "M", "M", "F", "F", "M"],
        }
    )
    result = sqldf(sql)
    return result


print(simulate_database_operation("SELECT * FROM my_table WHERE age > 50"))

              name  age sex
0      Henry Myers   60   F
1      Kelsey Lutz   54   M
2   Jonathan Young   76   F
3  Kimberly Macias   69   M


In [42]:
# from langchain_community.tools import DuckDuckGoSearchResults

# DuckDuckGoSearchResults().invoke("Obama")

In [43]:
from langchain.tools import tool


@tool
def simulate_database_operation(sql: str):
    """根据sql语句操作数据库"""
    my_table = pd.DataFrame(
        {
            "name": [
                "Henry Myers",
                "Martha Hawkins",
                "Kelsey Lutz",
                "Jonathan Fowler",
                "Jonathan Young",
                "Autumn Johnson",
                "Kimberly Macias",
                "Jared Mccormick",
                "Casey Hoover",
                "Erica Morse",
            ],
            "age": [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
            "sex": ["F", "M", "M", "F", "F", "M", "M", "F", "F", "M"],
        }
    )
    result = sqldf(sql)
    return result

## bind tools

In [44]:
tools = [DuckDuckGoSearchRun(), simulate_database_operation]
model_with_tools = model.bind_tools(tools)

In [ ]:
from pprint import pprint as pp

response = model_with_tools.invoke("印度的首都是哪里？")
pp(dict(response))

{'additional_kwargs': {'refusal': None},
 'content': '印度的首都是新德里（New '
            'Delhi）。它是印度联邦的首都，也是德里联邦属地的首府。新德里不仅是印度的政治中心，也是重要的文化和商业中心。',
 'example': False,
 'id': 'run-5c283fb1-2d41-4412-9d6a-8bcb7d0f0a96-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': '20250517114541b731985f8599439b',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 38,
                                       'completion_tokens_details': None,
                                       'prompt_tokens': 275,
                                       'prompt_tokens_details': None,
                                       'total_tokens': 313}},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_token_details': {},
                    'input_tokens': 275,
                    'output_token_de

In [ ]:
fetch_response = model_with_tools.invoke("现在是北京时间几点？查询之后告诉我")
pp(dict(fetch_response))

{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query": '
                                                                 '"北京时间现在几点"}',
                                                    'name': 'duckduckgo_search'},
                                       'id': 'call_-8724717192772033319',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-2f7db50b-7eed-4994-b921-c071c4fad946-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': '20250517114605c2f46f46a6f84d3a',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 14,
                                       'completion_tokens_details': None,
    

In [ ]:
db_response = model_with_tools.invoke(
    "帮我往数据库的my_table表中插入一条数据，name是张三，age是18，sex是male"
)
pp(dict(db_response))

{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"sql": '
                                                                 '"INSERT INTO '
                                                                 'my_table '
                                                                 '(name, age, '
                                                                 'sex) VALUES '
                                                                 "('张三', 18, "
                                                                 '\'male\')"}',
                                                    'name': 'simulate_database_operation'},
                                       'id': 'call_-8724732139261252859',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-55b69fb4-5f4a-4308-9c68-f894de7a3966-0',
 'invalid_tool_calls': [],
 'name': None

# agent

# manual agent

In [ ]:
from icecream import ic


def manual_agent(query: str, model: ChatOpenAI, tools: list[tool]):
    model_with_tools = model.bind_tools(tools)
    model_output = model_with_tools.invoke(query)
    tool_response = call_tool(model_output, tools)
    final_response = model.invoke(
        f"original query: {query} \n\n\n tool response: {tool_response}",
    )
    return final_response


def call_tool(model_output, tools):
    tools_map = {tool.name.lower(): tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool["name"]
        tool_args = tool["args"]
        tool_instance = tools_map[tool_name]
        tool_response = tool_instance.invoke(*tool_args.values())
        tools_response[tool_name] = tool_response
    return tools_response

In [50]:
manual_agent('帮我查询数据库my_table表中有多少人年龄大于60', model, tools).content


'根据您提供的查询结果，数据库`my_table`表中年龄大于60岁的人数是2人。这个结果是通过执行SQL查询`COUNT(*)`得到的，其中`COUNT(*)`是一个聚合函数，用于计算表中的行数。在这个上下文中，它计算了年龄大于60岁的记录数。'